### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime

# Load the NASA_API_KEY from the .env file
load_dotenv()
api_key = os.getenv('NASA_API_KEY')  # Assign the key to a consistent variable

# Validate the API key
if not api_key:
    raise ValueError("API Key not found in .env file!")


### CME Data

In [3]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url_cme = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={api_key}"

# Display the constructed URL (for debugging purposes)
print(f"Constructed CME Query URL: {query_url_cme}")



Constructed CME Query URL: https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2024-05-01&api_key=tshiwgqvXTnvIwBrx4YKzzQkZchnc483b98wm7PB


In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url_cme)

# Check if the request was successful
if cme_response.status_code == 200:
    print("CME data successfully retrieved!")
else:
    # Raise an error if the request fails
    raise ValueError(f"Failed to retrieve CME data. Status Code: {cme_response.status_code}")



CME data successfully retrieved!


In [5]:
# Convert the response variable to JSON and store it as a variable named cme_json
cme_json = cme_response.json()

# Preview the first few entries in the JSON response for verification
print(json.dumps(cme_json[:5], indent=4))


[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [6]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))



[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [7]:
# Convert cme_json to a Pandas DataFrame
cme_df = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]

# Preview the DataFrame
cme_df.head()



,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [8]:
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset=['linkedEvents'], inplace=True)

# Preview the updated DataFrame
cme_df.head()



,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [9]:
# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index, row in cme_df.iterrows():
    # Get the corresponding values from the row
    activity_id = row['activityID']
    start_time = row['startTime']
    linked_events = row['linkedEvents']

    # Iterate over each dictionary in the linkedEvents list
    for event in linked_events:
        # Append a new dictionary to the expanded_rows list for each event
        expanded_rows.append({
            'activityID': activity_id,
            'startTime': start_time,
            'linkedEvent': event.get('activityID')  # Extract 'activityID' from each dictionary in linkedEvents
        })

# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(expanded_rows)

# Preview the new DataFrame
expanded_cme_df.head()


,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T04:12:00-SEP-001


In [10]:
# Create a function called extract_activityID_from_dict
def extract_activityID_from_dict(input_dict):
    try:
        # Attempt to extract the 'activityID' from the dictionary
        return input_dict.get('activityID')
    except (AttributeError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error extracting activityID: {e}")
        return None

# Verify the function works as expected using one row from linkedEvents as an example
example_dict = expanded_cme_df['linkedEvent'][0]  # Example from the expanded DataFrame
print(extract_activityID_from_dict({'activityID': example_dict}))





2013-05-04T04:52:00-IPS-001


In [11]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column
# and create a new column called 'GST_ActivityID' using loc indexer
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict({'activityID': x}))

# Preview the updated DataFrame
expanded_cme_df.head()



,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,2013-05-04T04:52:00-IPS-001,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,2013-05-07T04:37:00-IPS-001,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,2013-05-12T23:30:00-IPS-001,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T01:53:00-FLR-001,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T04:12:00-SEP-001,2013-05-13T04:12:00-SEP-001


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs
expanded_cme_df.dropna(subset=['GST_ActivityID'], inplace=True)

# Preview the updated DataFrame
expanded_cme_df.head()



,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,2013-05-04T04:52:00-IPS-001,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,2013-05-07T04:37:00-IPS-001,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,2013-05-12T23:30:00-IPS-001,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T01:53:00-FLR-001,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,2013-05-13T04:12:00-SEP-001,2013-05-13T04:12:00-SEP-001


In [13]:
# Print out the datatype of each column in this DataFrame
print(expanded_cme_df.dtypes)



activityID        object
startTime         object
linkedEvent       object
GST_ActivityID    object
dtype: object


In [14]:
# Convert the 'GST_ActivityID' column to string format
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
expanded_cme_df.drop(columns=['linkedEvent'], inplace=True)

# Verify that all steps were executed correctly
expanded_cme_df.head()


,cmeID,startTime_CME,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01 03:12:00+00:00,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03 22:36:00+00:00,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09 19:29:00+00:00,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T04:12:00-SEP-001


In [15]:
# We are only interested in CMEs related to GSTs, so keep only rows where the GST_ActivityID column contains 'GST'
filtered_cme_df = expanded_cme_df[expanded_cme_df['GST_ActivityID'].str.contains('GST', na=False)]

# Verify that the filtering was successful
filtered_cme_df.head()



,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [16]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate = "2024-05-01"

# Build URL for GST
query_url_gst = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={api_key}"

# Display the constructed URL (for debugging purposes)
print(f"Constructed GST Query URL: {query_url_gst}")



Constructed GST Query URL: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=tshiwgqvXTnvIwBrx4YKzzQkZchnc483b98wm7PB


In [17]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_gst)

# Check if the request was successful
if gst_response.status_code == 200:
    print("GST data successfully retrieved!")
else:
    # Raise an error if the request fails
    raise ValueError(f"Failed to retrieve GST data. Status Code: {gst_response.status_code}")


GST data successfully retrieved!


In [18]:
# Convert the response variable to JSON and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[:1], indent=4))



[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    }
]


In [19]:
# Convert gst_json to a Pandas DataFrame
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: gstID, startTime, and linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]

# Preview the DataFrame
gst_df.head()





,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [20]:
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df.dropna(subset=['linkedEvents'], inplace=True)

# Preview the updated DataFrame
gst_df.head()



,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [21]:
# Use the explode method to ensure that each row is one element
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

# Drop any rows with missing values in 'linkedEvents' after the explosion
gst_df.dropna(subset=['linkedEvents'], inplace=True)

# Preview the updated DataFrame
gst_df.head()



,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [22]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column
# and create a new column called 'CME_ActivityID' using loc indexer
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: x.get('activityID') if isinstance(x, dict) else None)

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs
gst_df.dropna(subset=['CME_ActivityID'], inplace=True)

# Preview the updated DataFrame
gst_df.head()



,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [23]:
# Convert the 'CME_ActivityID' column to string format
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format
gst_df['gstID'] = gst_df['gstID'].astype(str)

# Convert startTime to datetime format
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])

# Rename startTime to startTime_GST
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df.head()


,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001


In [26]:
# Keep only rows where the CME_ActivityID column contains 'CME'
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]

# Preview the filtered DataFrame
gst_df.head()


,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [30]:
# Rename 'activityID' to 'cmeID' in cme_df
cme_df.rename(columns={'activityID': 'cmeID'}, inplace=True)

# Merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'cmeID' for cme
merged_df = pd.merge(
    gst_df,  # GST DataFrame
    cme_df,  # CME DataFrame
    left_on='CME_ActivityID',  # Match on 'CME_ActivityID' from GST
    right_on='cmeID',          # Match on 'cmeID' from CME
    how='inner'                # Perform an inner join
)

# Preview the merged DataFrame
merged_df.head()


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime,linkedEvents
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02T20:24Z,[{'activityID': '2013-06-07T03:00:00-GST-001'}]
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29T22:40Z,[{'activityID': '2013-09-30T05:05:00-SEP-001'}...
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04T23:12Z,[{'activityID': '2013-12-08T00:00:00-IPS-001'}...
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16T14:15Z,[{'activityID': '2014-02-18T05:00:00-IPS-001'}...
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18T01:25Z,[{'activityID': '2014-02-20T02:50:00-IPS-001'}...


In [31]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"Number of rows in GST DataFrame: {len(gst_df)}")
print(f"Number of rows in CME DataFrame: {len(cme_df)}")
print(f"Number of rows in Merged DataFrame: {len(merged_df)}")

# Check for any discrepancies
if len(merged_df) != len(gst_df) or len(merged_df) != len(cme_df):
    print("Warning: The merged DataFrame does not match the row count of the original DataFrames.")
else:
    print("Success: The merged DataFrame matches the row count of the original DataFrames.")



Number of rows in GST DataFrame: 61
Number of rows in CME DataFrame: 1023
Number of rows in Merged DataFrame: 61


### Computing the time it takes for a CME to cause a GST

In [35]:
# Ensure both startTime_GST and startTime are in datetime format
merged_df['startTime'] = pd.to_datetime(merged_df['startTime'])
merged_df['startTime_GST'] = pd.to_datetime(merged_df['startTime_GST'])

# Compute the time difference between startTime_GST and startTime (CME start time)
merged_df['timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime']).dt.total_seconds()

# Verify the new column
merged_df.head()


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime,linkedEvents,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,[{'activityID': '2013-06-07T03:00:00-GST-001'}],369360.0
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,[{'activityID': '2013-09-30T05:05:00-SEP-001'}...,188400.0
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,[{'activityID': '2013-12-08T00:00:00-IPS-001'}...,262080.0
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,[{'activityID': '2014-02-18T05:00:00-IPS-001'}...,218700.0
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,[{'activityID': '2014-02-20T02:50:00-IPS-001'}...,178500.0


In [36]:
# Use describe() to compute the mean and median time that it takes for a CME to cause a GST
time_stats = merged_df['timeDiff'].describe()

# Extract the mean and median values from the statistics
mean_time = time_stats['mean']
median_time = merged_df['timeDiff'].median()

# Print the mean and median time
print(f"Mean time for CME to cause a GST: {mean_time} seconds")
print(f"Median time for CME to cause a GST: {median_time} seconds")



Mean time for CME to cause a GST: 257366.55737704918 seconds
Median time for CME to cause a GST: 244260.0 seconds


### Exporting data in csv format

In [37]:
# Export the merged data to a CSV file without the index
merged_df.to_csv('merged_data.csv', index=False)

# Confirm the file has been saved
print("Data exported successfully to 'merged_data.csv'")



Data exported successfully to 'merged_data.csv'
